<h1> UOL Coursework Big Data</h1> 

Candidate number: RC0086<br>
Module code: DSM060<br>
Year 2021

The coursework is divided into two parts, Question 1 and Question 2. 
Question 1 is fully answered in this document. Question 2 also contains clustering output files which were added to the zip.

<h2> Question 1 </h2> <br> <h4>Find the maximum and minimum temperature of each day of a given month for the year 2007.</h4>

For question 1 I used the streaming technique which reads standard input and writes to standard output. To solve this task I broke the question into 2 parts.<br> Part 1 was to find the max and min temperature for a given station. Part 2 was to find the max and min temperature across all stations. 

Even though I tried to implement everything with one bigger mapper and a bigger reducer function, it failed at the map stage in Hadoop. Thus I solved it with a simpler, but more manual way as explained above. I also added the code of the bigger mapper at the end of Question 1, to show my attempt, but I am not sure as to why it did not work in Hadoop. 

In [5]:
#Part 1 of Question 1: FOR A GIVEN STATION, in this example 03011


#MAPPER

#! /usr/bin/env python3
import sys
import re

for line in sys.stdin:
   val = line.strip()
   (wban, ab, ac, ad, af, ag, ah, aj, ak, al, ae, ar, at, ay, au, ai, temp, ap, ax, av, an) = line.split(',')
   if (temp != '+9999' and re.match('[03011]', wban)):
       print("%s\t%s" % (wban, temp))
        

#REDUCER

#! /usr/bin/env python3


import sys

temperatures = []

for line in sys.stdin:
    (station, temp) = line.split('\t')

    temperatures.append(temp)

maxtemp = max(temperatures)
mintemp = min(temperatures)
 
print('For the station %s, the maximum temperature is %s, and the minimum temperature is %s' % (station, maxtemp, mintemp)) 
        
#output 
#For station "XXX", the maximum temperature is "XXX", and the minimum temperature is "XXX" 

ValueError: max() arg is an empty sequence

In [4]:
#Part 2 of Question 1: ACROSS ALL WEATHER STATIONS

#MAPPER

#! /usr/bin/env python3
import sys
import re

for line in sys.stdin:
   val = line.strip()
   (year, ab, ac, ad, af, ag, ah, aj, ak, al, ae, ar, at, ay, au, ai, temp, ap, ax, av, an) = line.split(',')
   if (temp != '+9999' and re.match('[03011]', year)):
       print("%s\t%s" % (year, temp))
        
        
#REDUCER

#! /usr/bin/env python3


import sys

temperatures = []

for line in sys.stdin:
    (station, temp) = line.split('\t')

    temperatures.append(temp)

maxtemp = max(temperatures)
mintemp = min(temperatures)
 
print('Across all weather stations, the maximum temperature is %s, and the minimum temperature is %s' % (maxtemp, mintemp)) 

#output 
#Across all weather stations, the maximum temperature is "XXX", and the minimum temperature is "XXX"

ValueError: max() arg is an empty sequence

In [ ]:
#Failed attemp at Question 1 with "all-in-one" Mapper Function
#The problem was, that it just did not work when transferring it to Hadoop.


lst = [i.split(',')[0:2] + [i.split(',')[8]] for i in lines[1:]]

station = set([i[0] for i in lst])

data = list(map(lambda stations: (max([l for l in lst if l[0] == stations]), min([l for l in lst if l[0] == stations])), station))

for collected_data in data:
    print(collected_data[0][1],collected_data[0][0],' max(',collected_data[0][2],')',' min(',collected_data[1][2],')')

    
#This was the supposed outcome which I had planned, however, it just did not work out in Hadoop, even though it works locally.

#20070401 03012  max( 37 )  min( 37 )
#20070401 03011  max( 45 )  min( 32 )

<h2> Question 2 </h2> <br> <h4>Create an implemention of k-means clustering.</h4>

For this task I used Mahout as it is tightly coupled with Hadoop. Additionally I used the dataset provided by the university "western-classics".

This document serves as the code representation along with the report. I used comments throughout, so the results should be easily reproducable by another person. 

In [3]:
#Getting Started with the dataset

#1 Create a new directory called "docs" using your command line 

mkdir ./docs 

#2 Uncompress the data and copy the dataset "western-classics" into your docs folder 
#  you can just drag and drop it using winscp
#  this dataset will represent the raw material on which the clustering algorithm will be tested

#3 Provide/Copy it to the Hadoop Distributed File System (HDFS) and you are ready to go

hadoop fs -copyFromLocal ./docs

#4 Converting the dataset into a SequenceFiles already using Mahoot 
#  A SequenceFile is a hadoop flat file format where each document is represented as key/value pair

mahout seqdirectory -i docs -o docs-seqfiles -c UTF-8 -chunk 5

#5 Convert the SequenceFiles to a sparse vector file format
#  this allows the text mining algorithm to work with a numerical representation of the text 
#  Therefore every document is represented as a vector where each element of the vector is a word and its weight

mahout seq2sparse -nv -i docs-seqfiles -o docs-vectors 

#6 Run Canopy Clustering on our data for grouping objects into clusters 
#  This allows us to set initial centroid values for our k-means algorithm in the next step

mahout canopy -i docs-vectors/tfidf-vectors -ow -o docs-vectors/docs-canopy-centroids -dm org.apache.mahout.common.distance.CosineDistanceMeasure -t1 1500 -t2 2000
mahout canopy -i docs-vectors/tfidf-vectors -ow -o docs-vectors/docs-canopy-centroids -dm org.apache.mahout.common.distance.ChebyshevDistanceMeasure -t1 1500 -t2 2000
mahout canopy -i docs-vectors/tfidf-vectors -ow -o docs-vectors/docs-canopy-centroids -dm org.apache.mahout.common.distance.EuclideanDistanceMeasure -t1 1500 -t2 2000

#7 Run k-means clustering algorithm with Mahout 
#  Don't forget to change the "userid" to your own, for me it is "klamm001"
#  Here we initialize the algorithm with "k = 10"

mahout kmeans -i docs-vectors/tfidf-vectors -c docs-canopy-centroids -o hdfs://lena/user/kklam001/docs-kmeans-clusters -dm org.apache.mahout.common.distance.CosineDistanceMeasure -cl -cd 0.1 -ow -x 20 -k 10    

#8 Print output of clustering and interpret it 
#  Don't forget to look-up the right path for your file "docs-kmeans-clusters/path" with hadoop fs -ls docs-kmeans-clusters [to find dir name]

mahout clusterdump -dt sequencefile -d docs-vectors/dictionary.file-* -i docs-kmeans-clusters/clusters-1-final -o clusters1.txt -b 100 -p docs-kmeans-clusters/clusteredPoints -n 20 --evaluate 

#9 Refresh (your winscp for example) and interpret the output

#10 use different "k-values" and "distance-measures" and repeat steps 1 to 9


SyntaxError: invalid syntax (<ipython-input-3-c154693a549a>, line 5)

<h3> Intro </h3>

This short report is a summary of a k-means clustering algorithm performed on a dataset provided by UOL. The main task was to carry out an analysis of these provided texts while changing parameters of the algorithm. The task was carried out on Apache Mahout and three different Distance Measures, with three different k-values each, were used. As a result Chebyshev Distance Measure was the best performing distance measure with the best k-value at around 10. 

K-Means <br>
This is an algorithm which partitions a dataset, in this case the british fiction corpus, into predefined "k" clusters.
The main goal is to create an intra-cluster which is as similar as possible. The algorithm works iteratively and achieves that the sum of the squared distances between the data points is at at a minimum. 


Text-Clustering <br>
Clustering in general is an technique to explore the structure of data. Text clustering is specified to the clustering of textual data. The main task is to identify similar subgroups in the text such as "top terms". The clustering is mostly done via different distance measures. The usage of a specific distance measure is application specific. Clustering is considered to be a unsupervised learning algorithm as it does not compare to "true" labels when evaluating the performance. 

Apache Mahout<br>
For this task Apache Mahout was used, which is a implementation of scalable machine learning algorithms focused on linear algebra and it uses the Apache Hadoop Platform. 


<h3> Methodology </h3>

The methodology was set out to create a clustering of the "british fiction corpus" dataset, using the kmeans algorithm on Apache Mahout. All the code for the implementation is provided above. 

In order to create a proper clustering we need to convert the text into vector space. In text mining the term frequency - inverse document frequency, also tf-idf, is a metric for evaluating the importance of a word in a document. The Vector Space model is used to represent textual information as a vector, so we can later use an algorithm to evaluate the text. This means that the VSM model shows the document in numerical vectors instead of its original textual state. 

For converting the text into vector space it is important to create a dictionairy of terms, which are in the document. Thus a index for the vocabulary is created and with the term frequency it is now possible to represent each term in the vector space. The represenation is quite simple as for every term (t1) we count the representation in the documents (d1). Then a representation is created where we see how often a term is in a certain document vector(d3)=(0,1,1,1). Finally it is possible to represent different documents and terms as a matrix with a |D| x F shape. In this case |D| is the amount of documents and F is the number of features (vocabulary). 

Now the clarification about the main concept has been done, the methodology should be explained in full. 

First of all we download the data from the UOL link and unzip it. For this exercise WinSCP and Putty was used. After unzipping the data we log into the lena hadoop environment of UOL, create a new directory and upload the data "british fiction corpus". The next step should be to copy the data into the Hadoop Distributed File System in order to handle it with mahout. After, the dataset has to be converted into a SequenceFile, which is a hadoop flat file format where each doc is shown as a key-value pair as shown above. Key is the "doc id" and value is the "content". The conversion in apache mahout is fullfilled by the seqdirectory command, where the text input is being converted. As we require a numerical representation for the k-means algorithm, we convert the sequenceFile into a sparse vector file. The process of converting into a vector space is explained above. After achieving the representation in the vector space we can run the k-means algorithm. You can also specify initial centroid values using canopy clustering before k-means. Last but not least after runnning the k-means algorithm an output file with the clustering result is generated which should be interpreted further. 

For this report 3 different distance measures were used: Cosine Distance, Euclidean Distance and Cherbyshev Distance. Eeach of these different distance measures was run with 3 different k-values: 5, 10 and 15. In the end 9 output files were generated with information about the generated clusters. The results are represented below in form of a table.  

<h3> Results </h3>



|                             | k = "5"                                                                              | k = "10"                                                                                  | k = "15"                                                                                  |
|-----------------------------|--------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------|
| Cosine Distance Measure     | Inter-Cluster Density: 0.46249913491703376    ........    Intra-Cluster Density: 0.6096840026831584 | Inter-Cluster Density: 0.4010578498181936    ........ Intra-Cluster Density: 0.5406987536756431 | Inter-Cluster Density: 0.4867190593999653     ........ Intra-Cluster Density: 0.5197197705550102 |
| Euclidean Distance Measure  | Inter-Cluster Density: 0.491557548792403     ........ Intra-Cluster Density: 0.5939504674570197   | Inter-Cluster Density: 0.5872501948175309     ........ Intra-Cluster Density: 0.5838604213524428 | Inter-Cluster Density: 0.5025281562946791     ........ Intra-Cluster Density: 0.5092799900117738 |
| Cherbyshev Distance Measure | Inter-Cluster Density: 0.4818027952402661     ........ Intra-Cluster Density: 0.5981898618172927  | Inter-Cluster Density: 0.35759758011425646     ........ Intra-Cluster Density: 0.618156306590354 | Inter-Cluster Density: 0.4041347767613779     ........ Intra-Cluster Density: 0.5650723335904658 |



Now it is possible to compare and interpret the results. The full files with the extracted words and created clusters are not shown here, however the Inter-Cluster Density and the Intra-Cluster Density, which are used for clustering quality assesment in this case. The Intra-Cluster Density should be greater then the Inter-Cluster Density, which is also the case in this analysis. Additionally, the best results indicate, when a high Intra-Cluster Density and a low Inter-Cluster Density ist shown. 

For this case, after some further analysis on calculating the mean, max and min of the values - it is clear that the Cherbyshev Distance Measure with a k value of 10 performed best on clustering. In that case the Inter-Cluster Density was around 0.3576 and the Intra-Cluster Density was around 0.6182 which is a decent value for such a task. 
For Intra-Cluster Density, Cherbyshev Distance has the greatest avarage, followed by Euclidean Distance. This indicates that Cherbyshev Distance Measure has a better cluster distribution, which shows the similarity of the instances in the cluster. Thus, the  smaller  the distance  between  instances, the  more  possible  instances  of the cluster, are  in the correct one.  For Inter-Cluster Density, Cherbyshev Distance has the smaller values, compared to the other distance measures. This states, that the clusters made with Cherbyshev Distance are more separeted, which indicates a difference. 
Finally, in terms of overall cluster quality the Cherbyshev Distance Measure performes best, followed by Cosine Distance. 

Considering limitations of mapreduce and the hadoop environment, the main concerns would pose the processing times. Even though some might have written a automation script, it is not always possible, and still the processing time of the algorithm is very high because it is processing a lot of small files. Moreover this was a smaller task, if there would have been an analysis of a huge dataset, the processing would be especially time consuming, which is definetely a major limitaion if you want to generate quick insight into the problem. Even if the Hadoop Environment helps, the files still need to be uploaded to HDFS, which again poses a difficult task due to memory if a lot of small files are uploaded. However, this can be overcome by converting the txt file into a sequential file straight away. 

All in all, this report was an insightful exercise into understanding text clustering and the mapreduce framework. Even though an automation script was provided by one of the students, I did not use it, as I wanted to solve this coursework on my own. Nevertheless, insightful answers could be attained and interpreted with help of the references listed below. 



<h2> References & Word Count </h2>

The word count of the report comprises of around 1369 words without tables and references. 

1. T.  White, Hadoop:  The  Definitive  Guide,  4th  ed.  Sebastopol,  CA: O’Reilly Media, 2015.
2. L.  Sahu  and  B.  R.  Mohan, An  improved  K-means  algorithm  using modified   cosine   distance   measure   for   document   clustering   using Mahout  with  Hadoop, 9th  International  Conference  on  Industrial  and Information Systems (ICIIS), Gwalior, India, 2014, pp. 1–5.
3. A. Huang, Similarity measures for text document clustering, Proc. New Zeal.  Comput.  Sci.  Res.  Student  Conf., Christchurch,  New  Zealand, 2008, pp. 49–56,.
4. A.  Strehl,  E.  Strehl,  J.  Ghosh,  and  R.  Mooney, Impact  of  similarity measures  on  web-page  clustering, Workshop  on  Artificial  Intelligence for Web Search, Texas, USA, 2000, pp. 58–64.
5. A.  Rangrej,  S.  Kulkarni,  and  A.  V.  Tendulkar, Comparative  study  of clustering techniques  for  short  text documents, Proceedings of  the 20th international  conference  companion  on  World  Wide  Web, Hyderabad, India, 2011, pp. 111–112.
6. Apache, Apache Mahout: Scalable Machine Learning and Data Mining. [Online]. Available:   http://mahout.apache.org/.   [Accessed:   20-Mar-2017].
7. R. Kannan, S. Vempala, and A. Vetta.  On clusterings:  Good, bad andspectral.J. ACM, 51(3):497–515, May 2004.